### Cào urls của tất cả các jobs

In [22]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random

# Hàm lấy liên kết công việc từ một trang
def fetch_job_links_from_page(base_url, page, retries=3):
    job_links_list = []
    url = f"{base_url}tat-ca-viec-lam-trang-{page}-vi.html"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    for attempt in range(retries):
        try:
            print(f"Đang lấy dữ liệu từ URL: {url}")
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 404:  # Điều kiện dừng khi trang không tồn tại
                print(f"Trang {page} không tồn tại. Dừng lại.")
                return None
            response.raise_for_status()  # Kiểm tra trạng thái phản hồi khác ngoài 404
            soup = BeautifulSoup(response.text, 'html.parser')
            job_links = soup.select('h2 > .job_link')
            job_links_list = [job['href'] for job in job_links]
            if not job_links_list:
                print(f"Không tìm thấy liên kết công việc nào trên trang {page}. Dừng lại.")
                return None  # Không còn liên kết nào
            print(f"Lấy được {len(job_links_list)} liên kết công việc từ trang {page}")
            return job_links_list
        except requests.RequestException as e:
            print(f"Thử lần {attempt + 1} thất bại trên trang {page}: {e}")
            if attempt < retries - 1:
                time.sleep(random.uniform(2, 5))  # Chờ trước khi thử lại
            else:
                print(f"Không thể lấy liên kết từ trang {page} sau {retries} lần thử")
    return None

# Hàm lấy liên kết công việc từ nhiều trang sử dụng đa luồng
def get_all_job_links(base_url, start_page=1, max_workers=20):
    job_links = []
    page = start_page
    stop_fetching = False
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        while not stop_fetching:
            futures = []
            for i in range(max_workers):  # Tạo nhiều tác vụ cùng lúc
                futures.append(executor.submit(fetch_job_links_from_page, base_url, page))
                page += 1
            for future in as_completed(futures):
                result = future.result()
                if result:
                    job_links.extend(result)
                else:
                    stop_fetching = True  # Dừng khi không có liên kết nào được trả về
    return job_links

# Sử dụng ví dụ
if __name__ == "__main__":
    BASE_URL = "https://careerviet.vn/viec-lam/"
    job_links = get_all_job_links(BASE_URL)

    # Lưu liên kết vào tệp
    with open('job_links.txt', 'w', encoding='utf-8') as f:
        for link in job_links:
            f.write(link + '\n')


Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-1-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-2-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-3-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-4-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-5-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-6-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-7-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-8-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-9-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-10-vi.html
Đang lấy dữ liệu từ URL: https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-11-vi.html
Đang lấy dữ liệu từ URL: https

### Cào jobs từ URL trong file job_links

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

# Hàm để cào thông tin từ URL công ty (nếu có)
def fetch_company_details(company_url, headers):
    try:
        response = requests.get(company_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy tên công ty
        company_name = soup.select_one("h1.name").text.strip() if soup.select_one("h1.name") else "Not specified"

        # Lấy địa điểm công ty
        company_address = (
            soup.select_one("strong:contains('Địa điểm') + p").text.strip()
            if soup.select_one("strong:contains('Địa điểm') + p")
            else "Not specified"
        )

        # Lấy quy mô công ty (loại bỏ phần đầu "Quy mô công ty:")
        company_size_element = soup.select_one("li:has(.mdi-account-supervisor)")
        company_size = (
            company_size_element.text.replace("Qui mô công ty:", "").strip()
            if company_size_element else "Not specified"
        )

        # Lấy loại hình hoạt động (loại bỏ phần đầu "Loại hình hoạt động:")
        company_type_element = soup.select_one("li:has(.mdi-gavel)")
        company_type = (
            company_type_element.text.replace("Loại hình hoạt động:", "").strip()
            if company_type_element else "Not specified"
        )

        # Lấy đường dẫn website công ty
        company_website_element = soup.select_one("li:has(.mdi-link)")
        company_website = company_website_element.text.replace("Website: ", "").strip() if company_website_element else "Not specified"

        # Lấy số lượng người theo dõi
        follow_element = soup.select_one("#countFollowers")
        follower = follow_element.text if follow_element else "Not Found"

        # Lấy giới thiệu công ty
        company_intro_element = soup.select_one(".intro-section .main-text")
        company_intro = company_intro_element.text.strip() if company_intro_element else "Not specified"

        return {
            "Company Name": company_name,
            "Company Address": company_address,
            "Company Scale": company_size,
            "Company Type": company_type,
            "Company Website": company_website,
            "Follower": follower,
            "Company Introduction": company_intro
        }
    except requests.RequestException as e:
        print(f"Error fetching company details from '{company_url}': {e}")
        return {
            "Company Name": "Not specified",
            "Company Address": "Not specified",
            "Company Scale": "Not specified",
            "Company Type": "Not specified",
            "Company Website": "Not specified",
            "Follower": "Not specified",
            "Company Introduction": "Not specified"
        }

# Hàm cào dữ liệu từ một URL công việc
def crawl_single_job(job_url, headers):
    try:
        print(f"Crawling: {job_url}")
        response = requests.get(job_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        time.sleep(random.uniform(1, 3))  # Chờ ngẫu nhiên để tránh bị chặn

        # Lấy tên công việc
        job_name = soup.select_one('h1.title').text.strip() if soup.select_one('h1.title') else "Not specified"

        # Lấy thông tin lương
        salary_info = (
            soup.select_one("strong:contains('Lương') + p").text.strip()
            if soup.select_one("strong:contains('Lương') + p")
            else "Not specified"
        )

        # Lấy địa điểm
        location_elements = soup.select('.map p a')
        locations = ', '.join([loc.text.strip() for loc in location_elements]) if location_elements else "Not specified"

        # Lấy các thông tin khác
        experience = (
            soup.select_one("strong:contains('Kinh nghiệm') + p").text.strip()
            if soup.select_one("strong:contains('Kinh nghiệm') + p")
            else "Not specified"
        )
        job_level = (
            soup.select_one("strong:contains('Cấp bậc') + p").text.strip()
            if soup.select_one("strong:contains('Cấp bậc') + p")
            else "Not specified"
        )
        deadline = (
            soup.select_one("strong:contains('Hết hạn nộp') + p").text.strip()
            if soup.select_one("strong:contains('Hết hạn nộp') + p")
            else "Not specified"
        )
        update_date = (
            soup.select_one("strong:contains('Ngày cập nhật') + p").text.strip()
            if soup.select_one("strong:contains('Ngày cập nhật') + p")
            else "Not specified"
        )
        employment_type = (
            soup.select_one("strong:contains('Hình thức') + p").text.strip()
            if soup.select_one("strong:contains('Hình thức') + p")
            else "Not specified"
        )

        # Lấy loại ngành nghề
        briefcase_element = soup.select_one("li:has(.mdi-briefcase)")
        list_briefcase = ", ".join([a.text for a in briefcase_element.find_all("a")]) if briefcase_element else "Not Found"

        # Lấy danh sách phúc lợi
        welfare_elements = soup.select("ul.welfare-list li")
        welfare_list = ', '.join([welfare.text.strip() for welfare in welfare_elements]) if welfare_elements else "Not specified"

        # Lấy phần "Mô tả Công việc"
        job_description_section = soup.select("h2:contains('Mô tả Công việc') ~ *")
        job_description_list = []
        for section in job_description_section:
            if section.name == "p" and section.text.strip():
                job_description_list.append(section.text.strip())
            elif section.name == "ul":
                description_items = section.find_all("li")
                job_description_list.extend([item.text.strip() for item in description_items if item.text.strip()])
        job_description = '\n'.join(job_description_list) if job_description_list else "Not specified"

        # Lấy phần "Yêu Cầu Công Việc"
        job_requirements_section = soup.select("h2:contains('Yêu Cầu Công Việc') ~ *")
        job_requirements_list = []
        for section in job_requirements_section:
            if section.name == "p" and section.text.strip():
                job_requirements_list.append(section.text.strip())
            elif section.name == "ul":
                requirements_items = section.find_all("li")
                job_requirements_list.extend([item.text.strip() for item in requirements_items if item.text.strip()])
        job_requirements = '\n'.join(job_requirements_list) if job_requirements_list else "Not specified"

        # Lấy nội dung của phần "Thông tin khác"
        other_info_section = soup.select_one("h3:contains('Thông tin khác') + div ul")
        other_info = '\n'.join([item.text.strip() for item in other_info_section.find_all("li")]) if other_info_section else "Not specified"

        # Lấy danh sách các Job Tags / Skills
        job_tags_elements = soup.select("div.job-tags ul li")
        job_tags = ', '.join([tag.text.strip() for tag in job_tags_elements]) if job_tags_elements else "Not specified"

        # Lấy URL công ty từ điều hướng
        company_url_element = soup.select_one("#tabs-job-company a")
        company_url = company_url_element['href'] if company_url_element else "Not Found"
        company_details = fetch_company_details(company_url, headers) if company_url != "Not Found" else {}

        # Tạo thông tin công việc
        job_information = {
            "Job Name": job_name,
            "Job Link": job_url,
            "Location": locations,
            "Update Date": update_date,
            "Deadline": deadline,
            "Experience": experience,
            "Job Level": job_level,
            "Industry": list_briefcase,
            "Employment Type": employment_type,
            "Welfare": welfare_list,
            "Job Description": job_description,
            "Job Requirements": job_requirements,
            "Other Information": other_info,
            "Job Tags": job_tags,
            "Company URL": company_url,
            **company_details,  # Thêm chi tiết công ty
            "Salary": salary_info
        }
        return job_information

    except requests.RequestException as e:
        print(f"Error fetching data from '{job_url}': {e}")
        return None

def crawl_job_information(job_urls):
    job_information_list = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    total_urls = len(job_urls)
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(crawl_single_job, url, headers) for url in job_urls]
        for i, future in enumerate(as_completed(futures), 1):
            result = future.result()
            if result:
                job_information_list.append(result)
            # Print progress
            print(f"Crawled {i}/{total_urls} URLs")
    
    return pd.DataFrame(job_information_list)


# Đọc các link từ file txt
with open('job_links.txt', 'r', encoding='utf-8') as f:
    job_links_from_file = [line.strip() for line in f]

# Crawl job information based on collected job links
job_df = crawl_job_information(job_links_from_file)

# Print the DataFrame
job_df

Crawling: https://careerviet.vn/vi/tim-viec-lam/a9-bvs-tuyen-tong-dai-vien-doi-tac-tap-doan-viettel-tai-pham-van-dong-nhan-sinh-vien-cho-bang-khong-yeu-cau-kinh-nghiem-viettel-ap.35C27247.htmlCrawling: https://careerviet.vn/vi/tim-viec-lam/a9-hoi-so-techcombank-tuyen-gap-nhan-vien-nhap-lieu-doi-tac-ngan-hang-techcombank-lam-thoi-vu-2-thang-khong-yeu-cau-kinh-nghiem.35C27245.html

Crawling: https://careerviet.vn/vi/tim-viec-lam/a9-tuyen-nhan-vien-tu-van-doi-tac-ngan-hang-singapore-tai-vu-pham-ham-lam-gio-hanh-chinh-khong-yeu-cau-kinh-nghiem.35C27244.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/a9-hoi-so-techcombank-tuyen-gap-nhan-vien-nhap-lieu-doi-tac-ngan-hang-techcombank-lam-thoi-vu-2-thang-lam-gio-hanh-chinh.35C27243.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/a9-tuyen-dung-nhan-vien-truc-tong-dai-ngan-hang-tpbank-khong-yeu-cau-kinh-nghiem-ho-tro-dau-thuc-tap-cho-sinh-vien.35C27242.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/hcm-talent-performance-lead.35C2

/home/yesin0918/miniconda3/envs/myenv/lib/python3.10/site-packages/soupsieve/css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Crawling: https://careerviet.vn/vi/tim-viec-lam/giao-dich-vien-kv-tay-ninh.35C1BEFE.html
Crawled 1/9880 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-chinh-nang-cao-trai-nghiem-khach-hang-ha-noi-ta085.35C1BF0A.html
Crawled 2/9880 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/xluxury-kien-truc-su-3d-thiet-ke-noi-that.35C1BF02.html
Crawled 3/9880 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-tu-van-tai-chinh-ca-nhan-tphcm-ta110.35C1C110.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/truong-ca-san-xuat-shift-in-charge.35C0D24D.html
Crawled 4/9880 URLs
Crawled 5/9880 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh.35C1BEE3.html
Crawled 6/9880 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/csr-manager-china-brand.35C1C10F.html
Crawled 7/9880 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh-khu-vuc-phia-bac.35C1BEEE.html
Crawled 8/9880 URLs
Crawling: https://careerviet.vn/vi/tim-viec-l

,Job Name,Job Link,Location,Update Date,Deadline,Experience,Job Level,Industry,Employment Type,Welfare,...,Job Tags,Company URL,Company Name,Company Address,Company Scale,Company Type,Company Website,Follower,Company Introduction,Salary
0,Strategic Portfolio Manager,https://careerviet.vn/vi/tim-viec-lam/strategi...,Hồ Chí Minh,18/10/2024,16/11/2024,Trên 5 \r\n ...,Trưởng nhóm / Giám sát,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Chế độ thưởng, Đào tạo, Tăng ...",...,"Purchasing, Nhân viên mua hàng Staff, Associa...",https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,CÔNG TY TNHH WABE SABE,"128 Trần Não, P.An Khánh, Q.2, TPHCM",10-20,Trách nhiệm hữu hạn,Not specified,18,Công ty thời trang Wabe sabe - Được ra đời từ ...,12 Tr - 15 Tr VND
1,Chăm Sóc Khách Hàng [Phú Mỹ - Bà Rịa Vũng Tàu],https://careerviet.vn/vi/tim-viec-lam/cham-soc...,Bà Rịa - Vũng Tàu,17/11/2024,17/12/2024,1 - 2 \r\n ...,Nhân viên,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Du lịch nước ngoài, ...",...,Not specified,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,Công Ty CP Liên Kết Quốc Tế INTERLINK (Trường ...,"27, đường CMT8, Phường Xuân An, Thị Xã Long Khánh",500-999,Cổ phần,https://inschool.edu.vn/,3.654,Tổ chức Giáo dục Interlink – thành lập từ năm ...,6 Tr - 8 Tr VND
2,Nhân viên kế toán kho,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,Hà Nội,17/11/2024,17/11/2024,1 - 3 \r\n ...,Nhân viên,\r\n ...,Nhân viên chính thức,"Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Đồn...",...,warehouse accountant,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,Công Ty Cổ Phần Tập Đoàn BNA,"Lô A2CN8 Cụm công nghiệp Từ Liêm, Phường Phươn...",1.000-4.999,Cổ phần,https://bnagroup.vn/,7.548,Công Ty Cổ Phần Tập Đoàn BNA bao gồm các thành...,9 Tr - 10 Tr VND
3,Trợ Giảng Tiếng Anh [Xuyên Mộc - Bà Rịa Vũng Tàu],https://careerviet.vn/vi/tim-viec-lam/tro-gian...,Bà Rịa - Vũng Tàu,17/11/2024,17/12/2024,Not specified,Nhân viên,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Du lịch nướ...",...,trợ giảng tiếng anh [xuyên mộc - bà rịa vũng t...,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,Công Ty CP Liên Kết Quốc Tế INTERLINK (Trường ...,"27, đường CMT8, Phường Xuân An, Thị Xã Long Khánh",500-999,Cổ phần,https://inschool.edu.vn/,3.654,Tổ chức Giáo dục Interlink – thành lập từ năm ...,7 Tr - 8 Tr VND
4,Trưởng Nhóm Tư Vấn Tuyển Sinh [Ngãi Giao - Bà ...,https://careerviet.vn/vi/tim-viec-lam/truong-n...,Bà Rịa - Vũng Tàu,17/11/2024,17/12/2024,Trên 1 \r\n ...,Trưởng nhóm / Giám sát,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Du lịch nước ngoài, ...",...,Not specified,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,Công Ty CP Liên Kết Quốc Tế INTERLINK (Trường ...,"27, đường CMT8, Phường Xuân An, Thị Xã Long Khánh",500-999,Cổ phần,https://inschool.edu.vn/,3.654,Tổ chức Giáo dục Interlink – thành lập từ năm ...,8 Tr - 12 Tr VND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9838,Nhân Viên Tư Vấn Website Quận 12 (Không Yêu Cầ...,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,Hồ Chí Minh,09/11/2024,30/11/2024,Not specified,Mới tốt nghiệp,\r\n ...,Nhân viên chính thức,"Laptop, Chế độ bảo hiểm, Du Lịch, Chế độ thưởn...",...,Not specified,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,CÔNG TY TNHH THƯƠNG MẠI VÀ DỊCH VỤ NINA,"Lầu 3, Tòa nhà SaigonTel, Lô 46, CVPM Quang Tr...",500-999,Trách nhiệm hữu hạn,Not specified,3,luôn định hướng thiết kế website chuyên nghiệp...,8 Tr - 26 Tr VND
9839,Giáo viên tiếng Anh TRẺ EM,https://careerviet.vn/vi/tim-viec-lam/giao-vie...,"Hà Nội, Thái Bình",09/11/2024,30/11/2024,Not specified,Mới tốt nghiệp,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...",...,giáo viên anh văn,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,Not specified,Not specified,Not specified,Not specified,Not specified,Not Found,Not specified,8 Tr - 15 Tr VND
9840,Nhân Viên Spa,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,Hà Nội,11/11/2024,14/12/2024,No

In [2]:
job_df.to_csv('job_data.csv', index=False, encoding='utf-8-sig')